In [18]:
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
from scipy import stats
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [116]:
#data from the past 15 days 

In [1]:
from alpha_vantage.timeseries import TimeSeries
from pprint import pprint
import pandas as pd
ts = TimeSeries(key='N1LNTR9K182JZQ3P',output_format='pandas', indexing_type='date')
data, meta_data = ts.get_intraday(symbol='AAPL',interval='1min', outputsize='full')
pd.set_option('display.max_rows', None)
data.head()

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2020-08-06 04:01:00,440.20,440.20,440.20,440.20,166.0
2020-08-06 04:09:00,439.21,439.21,439.21,439.21,307.0
2020-08-06 04:11:00,439.13,439.13,439.13,439.13,120.0
2020-08-06 04:16:00,439.17,439.17,439.13,439.13,703.0
2020-08-06 04:23:00,439.49,439.49,439.49,439.49,359.0


In [2]:
data['open'] =data['1. open'] 
data['close'] = data['4. close']
data['high'] = data['2. high']
data['low'] = data['3. low']
data = data.drop(columns=['1. open','4. close','2. high','3. low','5. volume'])
data = data.reset_index()
data.head()

,date,open,close,high,low
0,2020-08-06 04:01:00,440.20,440.20,440.20,440.20
1,2020-08-06 04:09:00,439.21,439.21,439.21,439.21
2,2020-08-06 04:11:00,439.13,439.13,439.13,439.13
3,2020-08-06 04:16:00,439.17,439.13,439.17,439.13
4,2020-08-06 04:23:00,439.49,439.49,439.49,439.49


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7520 entries, 0 to 7519
Data columns (total 5 columns):
date     7520 non-null datetime64[ns]
open     7520 non-null float64
close    7520 non-null float64
high     7520 non-null float64
low      7520 non-null float64
dtypes: datetime64[ns](1), float64(4)
memory usage: 293.8 KB


In [4]:
data['SR'] = (data.open - data.open.shift(1))/data.open.shift(1)

In [5]:
data = data.drop([0])
data.head()

,date,open,close,high,low,SR
1,2020-08-06 04:09:00,439.21,439.21,439.21,439.21,-0.002249
2,2020-08-06 04:11:00,439.13,439.13,439.13,439.13,-0.000182
3,2020-08-06 04:16:00,439.17,439.13,439.17,439.13,0.000091
4,2020-08-06 04:23:00,439.49,439.49,439.49,439.49,0.000729
5,2020-08-06 04:24:00,439.68,439.68,439.68,439.68,0.000432


In [7]:
yesterDay = data[(data['date'].dt.day==19)]
#yesterDay = yesterDay.drop([6825,6826,6827])
yesterDay.head()

,date,open,close,high,low,SR
6790,2020-08-19 04:01:00,463.88,463.88,463.88,463.88,0.002334
6791,2020-08-19 04:11:00,463.53,463.47,463.53,463.47,-0.000755
6792,2020-08-19 04:12:00,463.47,463.47,463.47,463.47,-0.000129
6793,2020-08-19 04:14:00,463.47,463.47,463.47,463.47,0.000000
6794,2020-08-19 04:22:00,463.93,463.47,463.93,463.47,0.000993


In [20]:
a = yesterDay.groupby(yesterDay['date'].dt.hour).mean()

In [9]:
xDay = data[data['date'].dt.day == 18 ]
xDay = xDay.drop(columns = ['open','close','high','low'])
xDay.head()

,date,SR
6084,2020-08-18 04:01:00,-0.000153
6085,2020-08-18 04:02:00,-0.000763
6086,2020-08-18 04:03:00,0.000000
6087,2020-08-18 04:04:00,0.001244
6088,2020-08-18 04:06:00,0.000109


In [21]:
b = xDay.groupby(xDay['date'].dt.hour).mean()

In [22]:
X = b[['SR']].astype(float)  # independent variables
Y = a[['close']].astype(float)  # target variable
linear_regressor = LinearRegression()  # create object for the class
linear_regressor.fit(X, Y)  # perform linear regression
print('Intercept: \n', linear_regressor.intercept_)
print('Coefficients: \n', linear_regressor.coef_)
# with statsmodels
X = sm.add_constant(X) # adding a constant
 
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

Intercept: 
 [464.43979754]
Coefficients: 
 [[8542.68653002]]
                            OLS Regression Results                            
Dep. Variable:                  close   R-squared:                       0.131
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     2.257
Date:                Thu, 20 Aug 2020   Prob (F-statistic):              0.154
Time:                        09:28:47   Log-Likelihood:                -31.759
No. Observations:                  17   AIC:                             67.52
Df Residuals:                      15   BIC:                             69.19
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------

C:\Users\umers\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\umers\Anaconda3\lib\site-packages\scipy\stats\stats.py:1416: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  "anyway, n=%i" % int(n))
